# Исследование надежности заемщиков.

**Заказчик** — кредитный отдел банка. 

**Цель исследования** 

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Входные данные**  — статистика о платёжеспособности клиентов.


**Результаты исследования** — На полученных данных будет произведено построение модели кредитного скоринга. (*специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.*)

## Обзор данных

Составляем первое представление о данных полученных от "кредитного отдела банка"

Добавление библиотеки

In [1]:
import pandas as pd

Чтение файла и его сохранение

In [2]:
df = pd.read_csv('datasets/data.csv')

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Описание таблицы**  <a id='info_DB'></a>

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

**Явные проблемы**
- `days_employed` - есть отрицательные значения, есть человек со стажем 340266 дней = 932 года, что явно ошибка. Вариант проверки, стаж не должен быть больше возраста, равен ему или различаться менее чем на 10 лет. Либо делать первичную проверку, что стаж более 21900 дней (60 лет стажа) скорее всего будет являться ошибкой (либо крайне редким исключением).
- `education` - разный регистр

**Неявные проблемы**
- Указание возраста с ошибкой (слишком маленький или большой)
- Указание зарплаты

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видимых проблем в названии таблиц нет.

В столбце `days_employed` и `total_income` меньше данных чем в остальных столбцах, скорее всего имеются `NULL` значения. Так же тип данных столбцов **float**, логичнее дни вести в формате целых чисел и для этого необходимо перевести в формат **int**



**Выводы**

В представленной таблицы - строки с данными о клиенте, его семейном положении, доходах и иная личная информация. Более подробно можно посмотреть значение столбцов тут -> [Расшифровка столбцов](#info_DB)

Предварительно можно утверждать, что, данных достаточно для проверки гипотез, но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

1. В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце вы обработаете на следующем этапе. Найдите другой столбец и заполните пропущенные значения в нём медианным значением по столбцу:
* опишите, какие пропущенные значения вы обнаружили;
* проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
* приведите возможные причины появления пропусков в данных;
* объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

2. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных. После обработки аномалий заполните пропуски в days_employed медианными значениями по этому столбцу.

3. Замените вещественный тип данных в столбце total_income на целочисленный, например, с помощью метода astype().

4. Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к одному регистру. Проверьте остальные столбцы.

5. После удаления дубликатов сделайте следующее:
* поясните, как выбирали метод для поиска и удаления дубликатов в данных;
* приведите возможные причины появления дубликатов.

6. Создайте два новых датафрейма, в которых:
* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

7. Удалите из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы — это те самые «словари» (не путайте с одноимённой структурой данных в Python), к которым вы сможете обращаться по идентификатору.

8. На основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
*Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.*

9. Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

*Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.*

Вы можете использовать собственную функцию и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.

### Заполнение пропусков

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце вы обработаете на следующем этапе. Найдите другой столбец и заполните пропущенные значения в нём медианным значением по столбцу:
* опишите, какие пропущенные значения вы обнаружили;
* проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
* приведите возможные причины появления пропусков в данных;
* объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

#### Основная проверка

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Есть 2174 пропущенных значений в столбцах `days_employed ` (общий трудовой стаж в днях) и `total_income` (ежемесячный доход).

Данные показатели могут быть важными значениями для модели кредитного скоринга

In [6]:
count_all = 21525

count_null_days = df['days_employed'].isna().sum() #подсчет количества нулевых значений в "общий трудовой стаж в днях"
count_null_income = df['total_income'].isna().sum() # #подсчет количества нулевых значений в "ежемесячный доход"

print('Количество пропусков значений:')
print('  "days_employed" {:.2%}'.format(count_null_days/count_all))
print('  "total_income"  {:.2%}'.format(count_null_income/count_all))


Количество пропусков значений:
  "days_employed" 10.10%
  "total_income"  10.10%


По хорошему нужно было бы написать запрос/функцию, которая бы прошлась по всем столбцам и брала не вписанное значение ручками, а максимальное в таблице

In [7]:
display(df[df['days_employed'].isna() & df['total_income'].isna()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Количество строк совпадает, можно сделать предположение, что `total_income` (ежемесячный доход) не заполняется, когда у человека отсутствует `days_employed ` (общий трудовой стаж в днях) или, например нет документального подтверждения официального трудоустройства. Данный момент лучше уточнить заказчика *(если есть возможность)*

In [8]:
# df['total_income'] = df['total_income'].fillna(df['total_income'].mean()) #заполнение пустых (среднее значение)

df['total_income'] = df['total_income'].fillna(df['total_income'].median())

# так же возможно произвести замену в зависимости от типа занятости
#df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))


In [9]:
display(df[df['days_employed'].isna()]) #проверим, заполнились ли данные

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,145017.937533,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,145017.937533,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,145017.937533,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,145017.937533,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,145017.937533,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,145017.937533,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,строительство жилой недвижимости


Использование медианной выборки позволит избежать влияния завышенных или заниженных данных на выборку

Когда у кого-то мясо, у кого-то капуста, а все мы едим голубцы ;)

Это могут быть люди как без опыта работы, так и без оффициального стажа. ПО этйо причине просто проставить 0 будет не совсем корректно. 


#### Дополнительная проверка  
<a id='check'></a>

In [10]:
df['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Есть дубликаты отличающиеся регистром, необходимо привести к общему виду.

In [11]:
df['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

Есть различие в регистре у 'Не женат / не замужем' и отличается от остальных. Однако можно оставить так как не встречаются совпадения. Скорее всего выбирается из предложенных вариантов.

In [12]:
df['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [13]:
df['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Цели получения кредита различаются по написанию, имеются ошибки в написании или порядке, но имеются общие "идеи".

In [14]:
df.duplicated().sum()

54

### Проверка данных на аномалии и исправления.

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных. После обработки аномалий заполните пропуски в days_employed медианными значениями по этому столбцу.

Проверка максимальных значений, минимальных через функцию `describe()`

In [15]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Явные проблемы**
* `children` - есть отрицательное значение
* `days_employed` - есть слишком большие данные 401755.400475, есть отрицательные значения
* `dob_years` - есть минимальная дата рождения 0

**Неявные проблемы**
* `children` - 20 детей может быть ошибкой (лишний ноль), но так же вполне может быть. По хорошему необходимо проверить как часто встречается более 10 детей.


In [16]:
df['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Можно заметить, что после 5 идет скачек сразу до 20 детей. Скорее всего может быть ошибкой. Так как в России считается рекордом такое количество детей.

In [17]:
df['children'] = df['children'].replace(-1, 1)
children_median = df[df['children'] != 20]['children'].median()
children_mean = df[df['children'] != 20]['children'].mean()
# df['children'] = df[df['children'] == 20] = children_median


print('медианное значение', children_median) # проверяем медианное значени
print('среднее значение', children_mean) # проверяем среднее значение и выбираем оптимальное

try:
    for count in df['children']:
        df['children'] = df['children'].replace(20, children_median)

except:
    print('отствуют строки в столбце children = 20')

    
print()
print(df['children'].value_counts().sort_index())
print()
df.info()

медианное значение 0.0
среднее значение 0.474334467807357

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Убрали отрицательное значение. Заменили значение 20 детей на медианное. Однако у нас из формата int превратился в float64. Нужно будет заменить

In [18]:
df['days_employed'].value_counts().sort_index()

-18388.949901     1
-17615.563266     1
-16593.472817     1
-16264.699501     1
-16119.687737     1
                 ..
 401663.850046    1
 401674.466633    1
 401675.093434    1
 401715.811749    1
 401755.400475    1
Name: days_employed, Length: 19351, dtype: int64

In [19]:
def minus_plus(ind):
    if ind < 0:
        ind *= -1
        return ind
    
    return ind

df['days_employed'] = df['days_employed'].apply(minus_plus)


days_employed_median = df['days_employed'].median() # со всеми значениями
days_employed_mean = df['days_employed'].mean()

days_employed_median2 = df[df['days_employed'] < 21900]['days_employed'].median() # с исключением "ошибочных" данных, более 21900 "дней"
days_employed_mean2 = df[df['days_employed'] < 21900]['days_employed'].mean()


print(df['days_employed'])
print()
print('Медианное значение со всеми данными', days_employed_median)
print('Среднее значение со всеми данными', days_employed_mean) # среднее значение слишком сильно завышает данные
print()
print('Медианное значение без "завышенных" данных', days_employed_median2)
print('Среднее значение без "завышенных" данных', days_employed_mean2) # теперь среднее значение не учитывает завышенные данные. 


df['days_employed'] = df['days_employed'].fillna(days_employed_median) 

print()
print('Итоговые данные:')
print(df['days_employed'])



0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Медианное значение со всеми данными 2194.220566878695
Среднее значение со всеми данными 66914.72890682195

Медианное значение без "завышенных" данных 1630.0193809778218
Среднее значение без "завышенных" данных 2353.015931998879

Итоговые данные:
0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64


In [20]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

В результате исправлений: 
* были заполнены пропуски медианными значением с учетом всех данных
* исправлены значения с отрицательным знаком

Стоит обратиться к ***заказчику*** с вопросом о формировании значений в столбце "опыт работы". 

In [21]:
df['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Возможна замена 0 возраста на медианный возраст людей по группам (к примеру по количеству детей)

In [22]:
print(df['dob_years'].median()) # берем медианное значение, хоть и среднее очень близко
print(df['dob_years'].mean())


dob_years_median = df['dob_years'].median()

df['dob_years'] = df['dob_years'].replace(0, dob_years_median)

df['dob_years'].value_counts().sort_index()

42.0
43.29337979094077


19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    698
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [23]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Имеется одна значение с неопределенным полом, можем оставить как не влияющее на выборку (так как всего одно значение). 
Стоит поднять вопрос, чтобы это небыло технической ошибкой которая со временем может увеличиться

**Вывод**

В результате предобработки данных были выявлены ***явные*** и ***неявные*** проблемы.

Для их справления были проведены следующие операции:

`children`
* Убрали отрицательное значение. 
* Заменили значение 20 детей на медианное.

`days_employed`
* были заполнены пропуски медианными значением с учетом всех данных
* исправлены значения с отрицательным знаком

`dob_years`
* заменил 0 значения на медианное


### Изменение типов данных.

Замените вещественный тип данных в столбце total_income на целочисленный, например, с помощью метода astype().

In [24]:
df['children'] = df['children'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int32 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int32 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(4), int64(3), object(5)
memory usage: 1.6+ MB


Заменили данные в столбцах на основной тип. Соответствие или "подходимость" можно посмотреть по [исходным данным](#info_DB).

Возможно для `debt` лучше сделать "булев" тип.

In [26]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Удаление дубликатов.


Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к одному регистру. Проверьте остальные столбцы.

После удаления дубликатов сделайте следующее:
* поясните, как выбирали метод для поиска и удаления дубликатов в данных;
* приведите возможные причины появления дубликатов.

Проверка столбцов проводилась ранее [Проверка](#check)

In [27]:
df['education'] = df['education'].str.lower()
print('уровень образования клиента:')
print()
print(df['education'].value_counts())

уровень образования клиента:

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Привели к единобразию, теперь мы можем удалять дубликаты

In [28]:

display(df[df.duplicated()].sort_values(by = 'dob_years'))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,2194,23,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
19321,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
18328,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости
21281,1,2194,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости
6312,0,2194,30,среднее,1,женат / замужем,0,M,сотрудник,0,145017,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
13639,0,2194,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,автомобиль
20187,0,2194,65,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
9528,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
9604,0,2194,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы


In [29]:
df = df.drop_duplicates()
df.duplicated().sum()

0

Скорее всего дубликаты возникли как повтороная подача заявки. В зависимости от сбора информации/передачи. Либо человеческий фактор.

Так же вполне возможно совпадение параметров, а не явный дубликат так как в этой таблице нет четкого индетификатора пользователя. 

###  Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создайте два новых датафрейма, в которых:
* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

In [30]:
df_education = df[['education_id', 'education']]
df_family_status = df[['family_status_id', 'family_status']]

print('Новые таблицы с дубликатами')
print(df_education.head(10))
print()
print(df_family_status.head(10))


Новые таблицы с дубликатами
   education_id education
0             0    высшее
1             1   среднее
2             1   среднее
3             1   среднее
4             1   среднее
5             0    высшее
6             0    высшее
7             1   среднее
8             0    высшее
9             1   среднее

   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем


Создаем две новых таблицы

In [31]:
df_education = df_education.drop_duplicates().reset_index(drop=True)
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True)

print('Новые таблицы без дубликатов')
print(df_education)
print()
print(df_family_status)

Новые таблицы без дубликатов
   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


Удаляем дубликаты

In [32]:
df.drop('education', axis='columns', inplace=True)
df.drop('family_status', axis='columns', inplace=True)
print(df.columns)

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')


Удалили расшифровку статуса и оставили только id

### Категоризация дохода.

На основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

*Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.*

In [33]:
def total_income_category(income):
    if income >= 0 and income <= 30000:
        return 'E'
    if income >= 30001 and income <= 50000:
        return 'D'    
    if income >= 50001 and income <= 200000:
        return 'C'
    if income >= 200001 and income <= 1000000:
        return 'B'    
    return 'A'

df['total_income_category'] = df['total_income'].apply(total_income_category)
display(df.sort_values(by='total_income'))


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
14585,0,359219,57,1,0,F,пенсионер,1,20667,недвижимость,E
13006,0,369708,37,1,1,M,пенсионер,0,21205,заняться высшим образованием,E
16174,1,3642,52,1,0,M,сотрудник,0,21367,приобретение автомобиля,E
1598,0,359726,68,1,1,M,пенсионер,0,21695,на проведение свадьбы,E
14276,0,346602,61,1,0,F,пенсионер,0,21895,недвижимость,E
...,...,...,...,...,...,...,...,...,...,...,...
17178,0,5734,42,0,1,M,компаньон,0,1711309,сыграть свадьбу,A
20809,0,4719,61,1,4,F,сотрудник,0,1715018,покупка жилья для семьи,A
9169,1,5248,35,1,1,M,сотрудник,0,1726276,дополнительное образование,A
19606,1,2577,39,0,0,M,компаньон,1,2200852,строительство недвижимости,A


**Вывод**

Посмотрев на максимальные и манимальные значения визуально можно сделать вывод, что код работает правильно.

Как итог добавил новую категорию.

### Категоризация целей кредита.

Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

*Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.*

Вы можете использовать собственную функцию и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.

In [34]:
df['purpose'].sort_values().unique()

def purpose_category(purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'недвижим' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадь' in purpose:
        return 'проведение свадьбы'    
    if 'образов' in purpose:
        return 'получение образования'
    
df['purpose_category'] = df['purpose'].apply(purpose_category)
display(df[df['purpose_category'].isna()]['purpose']) #проверим столбец на наличие пустых строк чтобы проверять наполненность категории
display(df)


Series([], Name: purpose, dtype: object)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21521,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21522,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21523,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


Категоризация успешно выполнена, добавлен новый столбец.

**Выводы**

Предобработка данных выявила следующие проблемы:
* наличие пропусков в данных
* аномальные данные
* данные с логической ошибкой
* загруженность датафрейма
* различие регистра
* дубликаты

Я заполнил пропуски медианными значениями, избавился от данных с логической ошибкой, привел данные к общему виду, упростил датафрейм разбив на 3 таблицы. А также добавил несколько новых столбцов для упрощения вычисления.


### Ответы на вопросы.

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?


#### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Решение:

Проанализируем количество кредитов и количество закрытых кредитов в зависимости от колличества детей

In [35]:
analysis_children = df.groupby('children').agg({'debt':['sum', 'count']})
print(analysis_children)

          debt       
           sum  count
children             
0         1071  14166
1          445   4855
2          194   2052
3           27    330
4            4     41
5            0      9


Посчитаем в процентном соотношении просрочки в зависимости от количества детей.

In [36]:
analysis_children[('debt', 'result')] = analysis_children[('debt', 'sum')] / analysis_children[('debt','count')]*100
print(analysis_children.sort_values(by=('debt','result')))

          debt                 
           sum  count    result
children                       
5            0      9  0.000000
0         1071  14166  7.560356
3           27    330  8.181818
1          445   4855  9.165808
2          194   2052  9.454191
4            4     41  9.756098


##### Вывод:

По результатам проверки, предварительно можно сделать вывод:
* заемщики с 5 детьми не допускают просрочки. Однако из-за небольшого количества таких заемщиков, результат может быть занижен. (в случае увеличения общего количества заемщиков, может измениться)
* больше всего просрочек приходит на людей с 4 детьми. Однако и в этом случае небольшое количество таких заемщиков.
* заемщики без детей допускают существенно меньше просрочек, чем заемщики с 1 и 2 детьми.
* заёмщики с 3 детьми возвращаю кредиты вовремя охотнее

**Данные выводы предварительны и могут измениться в случае увеличения данных заемщиков с 3, 4 и 5 детьми**


#### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Решение:

Как и в предыдущем решении, сгруппировав по семейному положению

In [37]:

analysis_family = df.merge(df_family_status, on='family_status_id', how='left').groupby('family_status').agg({'debt':['sum', 'count']})
print(analysis_family)

                      debt       
                       sum  count
family_status                    
Не женат / не замужем  274   2810
в разводе               85   1195
вдовец / вдова          63    959
гражданский брак       388   4150
женат / замужем        931  12339


In [38]:
analysis_family[('debt', 'result')] = analysis_family[('debt', 'sum')] / analysis_family[('debt','count')]*100
print(analysis_family.sort_values(by=('debt','result')))

                      debt                 
                       sum  count    result
family_status                              
вдовец / вдова          63    959  6.569343
в разводе               85   1195  7.112971
женат / замужем        931  12339  7.545182
гражданский брак       388   4150  9.349398
Не женат / не замужем  274   2810  9.750890


In [39]:
pt_af = df.merge(df_family_status, on='family_status_id', how='left').pivot_table(index=['family_status'], values='debt', aggfunc= 'mean', fill_value='X')*100
print(pt_af.sort_values('debt'))

                           debt
family_status                  
вдовец / вдова         6.569343
в разводе              7.112971
женат / замужем        7.545182
гражданский брак       9.349398
Не женат / не замужем  9.750890


##### Вывод:

По результатам проверки, предварительно можно сделать вывод:
* заемщики с семейным положением `вдовец / вдова` так и `в разводе` допускают меньше просрочек.
* заемщики с семейным положением `Не женат / не замужем` так и `гражданский брак` допускают больше просрочек.
* несмотря на то, что количество заемщиков с семейным статусом `женат / замужем` больше остальных более чем в 3 раза, количество просрочек среднее и приближается к минимальным показателям.

**Данные выводы предварительны и могут измениться в случае увеличения данных заемщиков `вдовец / вдова` или `в разводе`**


#### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Решение:

In [40]:
analysis_income = df.groupby('total_income_category').agg({'debt':['sum', 'count']})
print(analysis_income)

                       debt       
                        sum  count
total_income_category             
A                         2     25
B                       356   5041
C                      1360  16015
D                        21    350
E                         2     22


In [41]:
analysis_income[('debt', 'result')] = analysis_income[('debt', 'sum')] / analysis_income[('debt','count')]*100
print(analysis_income.sort_values(by=('debt','result')))

                       debt                 
                        sum  count    result
total_income_category                       
D                        21    350  6.000000
B                       356   5041  7.062091
A                         2     25  8.000000
C                      1360  16015  8.492039
E                         2     22  9.090909


In [42]:
pt_tic = df.pivot_table(index=['total_income_category'], values='debt', aggfunc= 'mean', fill_value='X')*100
print(pt_tic.sort_values('debt'))

                           debt
total_income_category          
D                      6.000000
B                      7.062091
A                      8.000000
C                      8.492039
E                      9.090909


Категории зарплаты:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

##### Вывод:

По результатам проверки, предварительно можно сделать вывод:
* заемщики с заработной платой в районе 30001–50000 реже допускают просрочки
* заемщики c заработной платой в районе 0–30000 и 50001–200000 чаще допускают просрочки
* заемщиков категории "А" и "Е" очень мало для того, чтобы по их данным делать окончательные выводы
* наиболее стабильная категория "B"

**Данные выводы предварительны и могут измениться в случае увеличения данных заемщиков категории с заработной платой `0–30000` или `1000001 и выше`. Так же в результате подстановки медианного значения в незаполненные строки, могли нарушить результаты исследования у группы "С" `50001–200000`**

**возможно стоит разделить группу "С" на более мелкие подгруппы, (50.001-100.000, 100.001-200.000). Так как в данной категории большое количество заемщиков по сравнению с остальными категориями и в итоге может измениться результат проверки просрочки**



#### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

##### Решение:

In [43]:
analysis_purpose = df.groupby('purpose_category').agg({'debt':['sum', 'count']})
analysis_purpose[('debt', 'result')] = analysis_purpose[('debt', 'sum')] / analysis_purpose[('debt','count')]*100
print(analysis_purpose.sort_values(by=('debt','result')))


                         debt                 
                          sum  count    result
purpose_category                              
операции с недвижимостью  782  10811  7.233373
проведение свадьбы        186   2323  8.006888
получение образования     370   4013  9.220035
операции с автомобилем    403   4306  9.359034


In [44]:
pt_ap = df.pivot_table(index=['purpose_category'], values='debt', aggfunc= 'mean', fill_value='X')*100
print(pt_ap.sort_values('debt'))

                              debt
purpose_category                  
операции с недвижимостью  7.233373
проведение свадьбы        8.006888
получение образования     9.220035
операции с автомобилем    9.359034


##### Вывод:

По результатам проверки, предварительно можно сделать вывод:
* заемщики с при проведении `операции с недвижимостью` реже допускают просрочки. Это можно объяснить тем, что часть людей живет в этой квартире и боится потерять. Часть людей может брать кредит на время, используют как инвестицию, купили на старте, продали - такие люди более финансово грамотные и понимают стоимость просрочки.
* заемщики, которые берут кредит при `операции с автомобилем`, чаще всего допускают просрочки. Это может быть вызвано множеством факторов. Бывают случаи, когда заемщики разбивают кредитный автомобиль или берут автомобиль, который дорого обслуживать.
* немного удивительно, что заемщики, которые берут кредит на `получение образования` имеют относительно высокий процент просрочек. Возможно, связано с тем, что они либо теряют работу и потому учатся, либо начинают учиться так как хотят сменить место работы. 
* заемщиков, берущих кредит на `проведение свадьбы` меньше всего, но и процент просрочек чуть больше минимального

**Возможно стоит разделить `получение образования` на повышение квалификации и получение другого образования. Так как при повышении квалификации, человек продолжает работать и от этого зависит его будущая работа, так как при повышении квалификации часть может оплачивать работодатель, что так же положительно скажется на итоговом результате"**


#### Дополнительная Аналитика

Сделаем сводный датафрейм с разбитием статуса и категорие по количеству детей, чтобы посмотреть на общую картину.

In [45]:
full_df = df.merge(df_family_status, on='family_status_id', how='left').pivot_table(index=['family_status', 'purpose_category'], columns='children', values='debt', aggfunc= 'count', fill_value='X')
print('Общее количество заемщиков:')
display(full_df)
print()
full_df = df.merge(df_family_status, on='family_status_id', how='left').pivot_table(index=['family_status', 'purpose_category'], columns='children', values='debt', aggfunc= 'sum', fill_value='X')
print('Общее количество просрочек:')
display(full_df)
full_df = df.merge(df_family_status, on='family_status_id', how='left').pivot_table(index=['family_status', 'purpose_category'], columns='children', values='debt', aggfunc= 'mean')*100
print('Среднее количество просрочек:')
display(full_df)

Общее количество заемщиков:


children                                             0       1      2      3  \
family_status         purpose_category                                         
Не женат / не замужем операции с автомобилем     519.0    94.0   20.0    4.0   
                      операции с недвижимостью  1294.0   258.0   41.0    2.0   
                      получение образования      458.0   102.0   14.0    2.0   
в разводе             операции с автомобилем     195.0    70.0   14.0    1.0   
                      операции с недвижимостью   433.0   185.0   48.0   10.0   
                      получение образования      158.0    61.0   19.0      X   
вдовец / вдова        операции с автомобилем     201.0    11.0    6.0      X   
                      операции с недвижимостью   474.0    52.0   12.0    3.0   
                      получение образования      176.0    18.0    2.0    3.0   
гражданский брак      операции с автомобилем     284.0   108.0   35.0    5.0   
                      операции с недвижимостью   650.0   261.0   61.0   13.0   
                      получение образования      269.0    97.0   32.0    6.0   
                      проведение свадьбы        1538.0   534.0  215.0   32.0   
женат / замужем       операции с автомобилем    1662.0   691.0  324.0   50.0   
                      операции с недвижимостью  4259.0  1715.0  873.0  141.0   
                      получение образования     1596.0   598.0  336.0   58.0   

children                                           4    5  
family_status         purpose_category                     
Не женат / не замужем операции с автомобилем       X    X  
                      операции с недвижимостью   1.0    X  
                      получение образования      1.0    X  
в разводе             операции с автомобилем     1.0    X  
                      операции с недвижимостью     X    X  
                      получение образования        X    X  
вдовец / вдова        операции с автомобилем       X    X  
                      операции с недвижимостью   1.0    X  
                      получение образования        X    X  
гражданский брак      операции с автомобилем     1.0  1.0  
                      операции с недвижимостью   4.0    X  
                      получение образования        X    X  
                      проведение свадьбы         3.0  1.0  
женат / замужем       операции с автомобилем     8.0  1.0  
                      операции с недвижимостью  15.0  5.0  
                      получение образования      6.0  1.0


Общее количество просрочек:


children                                            0      1     2     3    4  \
family_status         purpose_category                                          
Не женат / не замужем операции с автомобилем     64.0   13.0   5.0   0.0    X   
                      операции с недвижимостью   96.0   29.0   3.0   1.0  1.0   
                      получение образования      51.0   10.0   1.0   0.0  0.0   
в разводе             операции с автомобилем     13.0    6.0   2.0   0.0  0.0   
                      операции с недвижимостью   30.0   13.0   3.0   1.0    X   
                      получение образования      13.0    2.0   2.0     X    X   
вдовец / вдова        операции с автомобилем     19.0    0.0   1.0     X    X   
                      операции с недвижимостью   22.0    4.0   2.0   0.0  0.0   
                      получение образования      12.0    3.0   0.0   0.0    X   
гражданский брак      операции с автомобилем     28.0   17.0   4.0   2.0  0.0   
                      операции с недвижимостью   51.0   32.0   8.0   0.0  0.0   
                      получение образования      35.0   18.0   6.0   1.0    X   
                      проведение свадьбы        118.0   51.0  12.0   5.0  0.0   
женат / замужем       операции с автомобилем    121.0   68.0  36.0   3.0  1.0   
                      операции с недвижимостью  279.0  122.0  72.0  11.0  2.0   
                      получение образования     119.0   57.0  37.0   3.0  0.0   

children                                          5  
family_status         purpose_category               
Не женат / не замужем операции с автомобилем      X  
                      операции с недвижимостью    X  
                      получение образования       X  
в разводе             операции с автомобилем      X  
                      операции с недвижимостью    X  
                      получение образования       X  
вдовец / вдова        операции с автомобилем      X  
                      операции с недвижимостью    X  
                      получение образования       X  
гражданский брак      операции с автомобилем    0.0  
                      операции с недвижимостью    X  
                      получение образования       X  
                      проведение свадьбы        0.0  
женат / замужем       операции с автомобилем    0.0  
                      операции с недвижимостью  0.0  
                      получение образования     0.0

Среднее количество просрочек:


children                                                0          1  \
family_status         purpose_category                                 
Не женат / не замужем операции с автомобилем    12.331407  13.829787   
                      операции с недвижимостью   7.418856  11.240310   
                      получение образования     11.135371   9.803922   
в разводе             операции с автомобилем     6.666667   8.571429   
                      операции с недвижимостью   6.928406   7.027027   
                      получение образования      8.227848   3.278689   
вдовец / вдова        операции с автомобилем     9.452736   0.000000   
                      операции с недвижимостью   4.641350   7.692308   
                      получение образования      6.818182  16.666667   
гражданский брак      операции с автомобилем     9.859155  15.740741   
                      операции с недвижимостью   7.846154  12.260536   
                      получение образования     13.011152  18.556701   
                      проведение свадьбы         7.672302   9.550562   
женат / замужем       операции с автомобилем     7.280385   9.840810   
                      операции с недвижимостью   6.550834   7.113703   
                      получение образования      7.456140   9.531773   

children                                                2          3  \
family_status         purpose_category                                 
Не женат / не замужем операции с автомобилем    25.000000   0.000000   
                      операции с недвижимостью   7.317073  50.000000   
                      получение образования      7.142857   0.000000   
в разводе             операции с автомобилем    14.285714   0.000000   
                      операции с недвижимостью   6.250000  10.000000   
                      получение образования     10.526316        NaN   
вдовец / вдова        операции с автомобилем    16.666667        NaN   
                      операции с недвижимостью  16.666667   0.000000   
                      получение образования      0.000000   0.000000   
гражданский брак      операции с автомобилем    11.428571  40.000000   
                      операции с недвижимостью  13.114754   0.000000   
                      получение образования     18.750000  16.666667   
                      проведение свадьбы         5.581395  15.625000   
женат / замужем       операции с автомобилем    11.111111   6.000000   
                      операции с недвижимостью   8.247423   7.801418   
                      получение образования     11.011905   5.172414   

children                                                 4    5  
family_status         purpose_category                           
Не женат / не замужем операции с автомобилем           NaN  NaN  
                      операции с недвижимостью  100.000000  NaN  
                      получение образования       0.000000  NaN  
в разводе             операции с автомобилем      0.000000  NaN  
                      операции с недвижимостью         NaN  NaN  
                      получение образования            NaN  NaN  
вдовец / вдова        операции с автомобилем           NaN  NaN  
                      операции с недвижимостью    0.000000  NaN  
                      получение образования            NaN  NaN  
гражданский брак      операции с автомобилем      0.000000  0.0  
                      операции с недвижимостью    0.000000  NaN  
                      получение образования            NaN  NaN  
                      проведение свадьбы          0.000000  0.0  
женат / замужем       операции с автомобилем     12.500000  0.0  
                      операции с недвижимостью   13.333333  0.0  
                      получение образования       0.000000  0.0

In [46]:
#код ревьюера
df.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children,,,
0,14166,1071,0.075604
1,4855,445,0.091658
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [47]:
#код ревьюера
children_pivot = df.pivot_table(index=['children'], values='debt', aggfunc=['sum', 'count'])
children_pivot = children_pivot['sum'] / children_pivot['count']
children_pivot

,debt
children,
0,0.075604
1,0.091658
2,0.094542
3,0.081818
4,0.097561
5,0.000000


**Занимательная статистика**
* На проведение свадьбы берут кредит только заемщики с категорией `гражданский брак`
* Заемщики с 3 детьми и более чаще всего берут кредит на `недвижимость`
* Больше всего кредитов у `женат / замужем` заемщиков без детей на `недвижимость` и просрочек так же больше всего.
* При примерно одинаковом количестве заемщиков `в разводе` и `вдовец / вдова` при `операции с автомобилем` и без детей, количество просрочек больше у `вдовец / вдова`



**Занимательная статистика v1**
* больше всего просрочек у `Не женат / не замужем` при `операции с автомобилем` с 2 детьми - 25% (не много данных)
* на втором месте количество просрочек у категории `гражданский брак` при `получение образования` с 2 детьми - 18.75%
* на третьем месте количество просрочек у категории `гражданский брак` при `получение образования` с 1 ребенком - 18.56%
* несмотря на то, что больше всего кредитов у `женат / замужем` заемщиков без детей на `недвижимость`, % просрочек маленький - 6.55%
* самый маленький % просрочки у заемщика `в разводе` с 1 ребенком при получении кредита на `получение образования` - 3.28% (но мало заемщиков всего 61)


## Общий вывод:

В результате исследования были выявлен ряд явных и неявных ошибок. Часть данных требует уточнения у заказчика. Подготовил дополнительные данные для облегчения обработки данных в дальнейшем.

В результате можно выделить положительные характеристики заемщика:

* отсутствие детей у заемщика повышает его кредитный статус

* наличие официального семейного статуса благоприятно влияет на выплату

* наиболее стабильная категория заемщиков с заработной платой "200001–1000000"

* заемщики с при проведении `операции с недвижимостью` реже допускают просрочки. Это можно объяснить тем, что часть людей живет в этой квартире и боится потерять. Часть людей может брать кредит на время, используют как инвестицию, купили на старте, продали - такие люди более финансово грамотные и понимают стоимость просрочки.


Для наиболее точной аналитики нужно: 
* собрать больше данных по заемщикам с 4 и 5 детьми. 
* Обратить внимание на категорию заемщиков с заработной платой от 0 до 30к - входит в ***группу риска***, при увеличении количества выдачи кредитов данной категории, может как уменьшиться общий процент просрочек, так и увеличиться.
*
